# Our recommendation system

In [ ]:
# Uncomment to install packages
# !pip install -r requirements.txt

## Generic MySQL to igraph methods

In [ ]:
PASSWORD = '<your_MYSQL_instance_password>'
import mysql.connector

connection = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd=PASSWORD,
  database="steam_data")

In [ ]:
# Uses the MySQL function we created in the previous step
from graphtastic.database.mysql import query_mysql

## Custom methods

In [ ]:
%%writefile graphtastic/igraph/helper.py
def create_igraph_ids(nodes, from_index=0):
	igraph_ids = {internal_id: igraph_id for igraph_id, internal_id
              	in enumerate(nodes, from_index)}
	return igraph_ids


In [ ]:
from graphtastic.igraph.helper import create_igraph_ids

In [ ]:
%%writefile graphtastic/database/igraph.py
import igraph as ig
from graphtastic.database.mysql import query_mysql
from graphtastic.igraph.helper import create_igraph_ids

def mysql_to_graph(table, source, target, weights, password):
    sql_query = f'SELECT {source}, {target}, {weights} FROM {table}'
    data = query_mysql(sql_query, password=password)
    source_nodes = sorted(list(set([source for source, _, _ in data])))
    target_nodes = sorted(list(set([target for _, target, _ in data])))
    source_igraph_ids = create_igraph_ids(source_nodes)
    target_igraph_ids = create_igraph_ids(target_nodes, len(source_igraph_ids))
    edges = [(source_igraph_ids[source], target_igraph_ids[target])
         	for source, target, _ in data]
    weights = [weight for _, _, weight in data]
    g = ig.Graph(directed=True)
    g.add_vertices(len(source_nodes + target_nodes))
    g.vs['internal_id'] = list(source_igraph_ids.keys()) + list(target_igraph_ids.keys())
    g.vs['type'] = ['source' for _ in source_nodes] + ['target' for _ in target_nodes]
    g.add_edges(edges)
    g.es['weight'] = weights
    return g

In [ ]:
from graphtastic.database.igraph import mysql_to_graph

In [ ]:
def prune_graph(g, min_hours):
	edges_to_remove = g.es.select(weight_lt=min_hours)
	g.delete_edges(edges_to_remove)
	return g

In [ ]:
def make_recommendations(g, user, min_hours):
    user_node = g.vs.select(internal_id_eq=user)
    user_node = user_node[0].index
    g = prune_graph(g, min_hours)
    other_user_nodes = g.vs.select(type_eq='source')
    pairs = [[user_node, other_user.index] for other_user in other_user_nodes if other_user.index != user_node]
    similarities = g.similarity_jaccard(pairs=pairs, mode='out')
    node_similarity = [[pair[1], similarity] for pair, similarity in zip(pairs, similarities)]
    node_similarity = sorted(node_similarity, key=lambda x: x[1], reverse=True)
    most_similar_node = node_similarity[0][0]
    game_recommendations = g.vs[g.neighbors(most_similar_node)]['internal_id']
    owned_games = g.vs[g.neighbors(user_node)]['internal_id']
    new_games = [game for game in game_recommendations if game not in owned_games]
    return new_games

In [ ]:
g = mysql_to_graph('steam_play', 'id', 'game_name', 'hours', password=PASSWORD)
recommendations = make_recommendations(g, '87907200', min_hours=0.5)
print(recommendations)